- Họ tên - mã số sinh viên:
  - Sinh viên 1: Võ Minh Trí - 19522396
  - Sinh viên 2: Vũ Hữu Tùng - 19522497
  - Sinh viên 3: Nguyễn Thị Mai Phương - 19522064 
- Môn: Xử Lý Ngôn Ngữ Tự Nhiên Cho Khoa Học Dữ Liệu 
- Tên Đồ Án Môn Học: Vietnamese Text Summarization

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

MessageError: ignored

## đọc dữ liệu

In [ ]:
import pandas as pd
df_train = pd.read_csv('/content/drive/MyDrive/Năm 3/NLP/Do_An/dataset/dataset/data_train.csv', encoding='utf-8')
df_dev = pd.read_csv('/content/drive/MyDrive/Năm 3/NLP/Do_An/dataset/dataset/data_val.csv', encoding='utf-8')
df_test = pd.read_csv('/content/drive/MyDrive/Năm 3/NLP/Do_An/dataset/dataset/test_data.csv', encoding='utf-8')


In [ ]:
#so sanh
set_test = pd.read_csv('/content/drive/MyDrive/Năm 3/NLP/Do_An/dataset/dataset/test_data.csv', encoding='utf-8')

In [ ]:
set_test = set_test.drop(['Unnamed: 0', 'file'], axis=1)

In [ ]:
# set_test = set_test[0:10000]

In [ ]:
df_train = df_train.drop(['Unnamed: 0', 'file'], axis=1)
df_dev = df_dev.drop(['Unnamed: 0', 'file'], axis=1)
df_test = df_test.drop(['Unnamed: 0', 'file'], axis=1)

In [ ]:
#df_train = df_train[0:50000]
#df_dev = df_dev[0:10000]
# data_20k = train_df[0:20000]
#df_test = df_test[0:10000]

In [ ]:
df_train = df_train[0:80000]
df_dev = df_dev[0:20000]
# data_20k = train_df[0:20000]
df_test = df_test[0:20000]
#df_test_1 = df_test[20000:22644]

In [ ]:
df_test.shape

(20000, 2)

In [ ]:
df_test.head(2)

,original,summary
0,"HMS Queen_Elizabeth thử_nghiệm trên biển hồi năm 2018 . Bộ_trưởng Quốc_phòng Anh Gavin_Williamson hôm_nay công_bố chiến_lược_quân_sự sau khi Anh rời Liên_minh châu_Âu ( EU ) , trong đó gồm kế_hoạc...","Bộ_trưởng Quốc_phòng Anh vạch ra chiến_lược hậu Brexit , đề_cao phô_trương sức_mạnh quân_sự để bảo_vệ lợi_ích của London ."
1,"Bộ_trưởng Công_Thương_Trần_Tuấn_Anh vừa kỷ_luật hạ bậc lương ông Bùi_Quang_Hưng - Giám_đốc Trung_tâm Hỗ_trợ xuất_khẩu ( thuộc Cục Xúc_tiến thương_mại ) vì đi lễ trong giờ_hành_chính . Cụ_thể , ông...","Nhận kỷ_luật sau khi đi lễ trong giờ_hành_chính , Giám_đốc trung_tâm hỗ_trợ xuất_khẩu ( Cục Xúc_tiến thương_mại ) bị hạ bậc lương . Hội_đồng_kỷ_luật cũng cảnh_cáo 7 cán_bộ cấp phòng , khiển_trách ..."


In [ ]:
df_train['summary'][1]

'USS Michigan , tàu_ngầm hạt_nhân Mỹ tới cảng Busan của Hàn_Quốc giữa khi căng_thẳng leo_thang trên bán_đảo Triều_Tiên . Động_thái này được cho là nhằm gia_tăng áp_lực lên Bình_Nhưỡng . '

## pretrained

In [ ]:
#Add sostok and eostok at 
# post_pre_train = df_train
# post_pre_dev = df_dev
# post_pre_test = df_test
df_train['summary'] = df_train['summary'].apply(lambda x : 'sostok '+ x + ' eostok')
df_dev['summary'] = df_dev['summary'].apply(lambda x : 'sostok '+ x + ' eostok')
df_test['summary'] = df_test['summary'].apply(lambda x : 'sostok '+ x + ' eostok')

In [ ]:
!pip install pyvi

In [ ]:
# Extract word vectors
import numpy as np
embeddings_index = {}
f = open('/content/drive/MyDrive/Năm 3/NLP/Do_An/Glove_attention_layer/glove.6B.100d.txt', encoding='utf-8')
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()

In [ ]:
from tensorflow.keras.preprocessing.sequence import pad_sequences
from pyvi import ViTokenizer
from keras.preprocessing.text import Tokenizer

max_len = 100

word_tokenizer = Tokenizer(oov_token=-1)
word_tokenizer.fit_on_texts(df_train.original.values.flatten())
word_to_index = word_tokenizer.word_index
word_to_index['pad'] = 0
word_to_index['unk'] = -1

index_to_word = {i: w for w, i in word_to_index.items()}

def encodingx(X):
    sentences = []
    
    for t in X:
        tokenized_sentence = ViTokenizer.tokenize(t)
        sentences.append(tokenized_sentence)
    
    X = word_tokenizer.texts_to_sequences(sentences)
    X = pad_sequences(maxlen = max_len, sequences = X, padding = "post", value = word_to_index['pad'])

    return X

In [ ]:
word_to_index['tôi']

86

In [ ]:
from tensorflow.keras.preprocessing.sequence import pad_sequences
from pyvi import ViTokenizer
from keras.preprocessing.text import Tokenizer

max_len = 100

yword_tokenizer = Tokenizer(oov_token=-1)
yword_tokenizer.fit_on_texts(df_train.summary.values.flatten())
yword_to_index = yword_tokenizer.word_index
yword_to_index['pad'] = 0
yword_to_index['unk'] = -1

yindex_to_word = {i: w for w, i in yword_to_index.items()}

def encodingy(X):
    sentences = []
    
    for t in X:
        tokenized_sentence = ViTokenizer.tokenize(t)
        sentences.append(tokenized_sentence)
    
    X = yword_tokenizer.texts_to_sequences(sentences)
    X = pad_sequences(maxlen = max_len, sequences = X, padding = "post", value = yword_to_index['pad'])

    return X

In [ ]:
# first create a matrix of zeros, this is our embedding matrix
import numpy as np

embedding_dim = 100
max_feature = len(embeddings_index) + 2
num_words = len(word_to_index)
embedding_matrixX = np.zeros((num_words, embedding_dim))

# for each word in out tokenizer lets try to find that work in our w2v model
for word, i in word_to_index.items():
    if i > max_feature:
        continue
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        embedding_matrixX[i] = embedding_vector
    else:
        # doesn't exist, assign a random vector
        embedding_matrixX[i] = np.random.randn(embedding_dim)

In [ ]:
# first create a matrix of zeros, this is our embedding matrix
ynum_words = len(yword_to_index)
embedding_matrixy = np.zeros((ynum_words, embedding_dim))

# for each word in out tokenizer lets try to find that work in our w2v model
for word, i in yword_to_index.items():
    if i > max_feature:
        continue
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        embedding_matrixy[i] = embedding_vector
    else:
        # doesn't exist, assign a random vector
        embedding_matrixy[i] = np.random.randn(embedding_dim)

In [ ]:
# X_train_encoded = encoding(X_train)
# X_dev_encoded = encoding(X_dev)
# X_test_encoded = encoding(X_test)

X_train_encoded = encodingx(df_train.original.values.flatten())
y_train_encoded = encodingy(df_train.summary)
X_dev_encoded = encodingx(df_dev.original.values.flatten())
y_dev_encoded = encodingy(df_dev.summary)
X_test_encoded = encodingx(df_test.original.values.flatten())
y_test_encoded = encodingy(df_test.summary)

In [ ]:
y_test_encoded[0]

array([   2,   35,  157,   19,  127,  141, 3026,   29,  161, 1044,  455,
       2504,  146,  125, 2409,  707,  518,  314,   85,   66,   24,  223,
        449,  505, 1249,    4, 1540,    3,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0], dtype=int32)

array([[   2,   33,  155, ...,    0,    0,    0],
       [   2,  114,  500, ...,    0,    0,    0],
       [   2,  122,  282, ...,    0,    0,    0],
       ...,
       [   2,  340, 1712, ...,    0,    0,    0],
       [   2,   39, 1328, ...,    0,    0,    0],
       [   2,  108,  536, ...,    0,    0,    0]], dtype=int32)

In [ ]:
#Adding Custom Attention layer 

import tensorflow as tf
import os
from keras.layers import Layer
from keras import backend as K


class AttentionLayer(Layer):
    def __init__(self, **kwargs):
        super(AttentionLayer, self).__init__(**kwargs)

    def build(self, input_shape):
        assert isinstance(input_shape, list)
        # Create a trainable weight variable for this layer.

        self.W_a = self.add_weight(name='W_a',
                                   shape=tf.TensorShape((input_shape[0][2], input_shape[0][2])),
                                   initializer='uniform',
                                   trainable=True)
        self.U_a = self.add_weight(name='U_a',
                                   shape=tf.TensorShape((input_shape[1][2], input_shape[0][2])),
                                   initializer='uniform',
                                   trainable=True)
        self.V_a = self.add_weight(name='V_a',
                                   shape=tf.TensorShape((input_shape[0][2], 1)),
                                   initializer='uniform',
                                   trainable=True)

        super(AttentionLayer, self).build(input_shape)  # Be sure to call this at the end

    def call(self, inputs, verbose=False):
        """
        inputs: [encoder_output_sequence, decoder_output_sequence]
        """
        assert type(inputs) == list
        encoder_out_seq, decoder_out_seq = inputs
        if verbose:
            print('encoder_out_seq>', encoder_out_seq.shape)
            print('decoder_out_seq>', decoder_out_seq.shape)

        def energy_step(inputs, states):
            """ Step function for computing energy for a single decoder state """

            assert_msg = "States must be a list. However states {} is of type {}".format(states, type(states))
            assert isinstance(states, list) or isinstance(states, tuple), assert_msg

            """ Some parameters required for shaping tensors"""
            en_seq_len, en_hidden = encoder_out_seq.shape[1], encoder_out_seq.shape[2]
            de_hidden = inputs.shape[-1]

            """ Computing S.Wa where S=[s0, s1, ..., si]"""
            # <= batch_size*en_seq_len, latent_dim
            reshaped_enc_outputs = K.reshape(encoder_out_seq, (-1, en_hidden))
            # <= batch_size*en_seq_len, latent_dim
            W_a_dot_s = K.reshape(K.dot(reshaped_enc_outputs, self.W_a), (-1, en_seq_len, en_hidden))
            if verbose:
                print('wa.s>',W_a_dot_s.shape)

            """ Computing hj.Ua """
            U_a_dot_h = K.expand_dims(K.dot(inputs, self.U_a), 1)  # <= batch_size, 1, latent_dim
            if verbose:
                print('Ua.h>',U_a_dot_h.shape)

            """ tanh(S.Wa + hj.Ua) """
            # <= batch_size*en_seq_len, latent_dim
            reshaped_Ws_plus_Uh = K.tanh(K.reshape(W_a_dot_s + U_a_dot_h, (-1, en_hidden)))
            if verbose:
                print('Ws+Uh>', reshaped_Ws_plus_Uh.shape)

            """ softmax(va.tanh(S.Wa + hj.Ua)) """
            # <= batch_size, en_seq_len
            e_i = K.reshape(K.dot(reshaped_Ws_plus_Uh, self.V_a), (-1, en_seq_len))
            # <= batch_size, en_seq_len
            e_i = K.softmax(e_i)

            if verbose:
                print('ei>', e_i.shape)

            return e_i, [e_i]

        def context_step(inputs, states):
            """ Step function for computing ci using ei """
            # <= batch_size, hidden_size
            c_i = K.sum(encoder_out_seq * K.expand_dims(inputs, -1), axis=1)
            if verbose:
                print('ci>', c_i.shape)
            return c_i, [c_i]

        def create_inital_state(inputs, hidden_size):
            # We are not using initial states, but need to pass something to K.rnn funciton
            fake_state = K.zeros_like(inputs)  # <= (batch_size, enc_seq_len, latent_dim
            fake_state = K.sum(fake_state, axis=[1, 2])  # <= (batch_size)
            fake_state = K.expand_dims(fake_state)  # <= (batch_size, 1)
            fake_state = K.tile(fake_state, [1, hidden_size])  # <= (batch_size, latent_dim
            return fake_state

        fake_state_c = create_inital_state(encoder_out_seq, encoder_out_seq.shape[-1])
        fake_state_e = create_inital_state(encoder_out_seq, encoder_out_seq.shape[1])  # <= (batch_size, enc_seq_len, latent_dim

        """ Computing energy outputs """
        # e_outputs => (batch_size, de_seq_len, en_seq_len)
        last_out, e_outputs, _ = K.rnn(
            energy_step, decoder_out_seq, [fake_state_e],
        )

        """ Computing context vectors """
        last_out, c_outputs, _ = K.rnn(
            context_step, e_outputs, [fake_state_c],
        )

        return c_outputs, e_outputs

    def compute_output_shape(self, input_shape):
        """ Outputs produced by the layer """
        return [
            tf.TensorShape((input_shape[1][0], input_shape[1][1], input_shape[1][2])),
            tf.TensorShape((input_shape[1][0], input_shape[1][1], input_shape[0][1]))
        ]

## build model

In [ ]:
from keras import backend as K 
import gensim
from numpy import *
import numpy as np
import pandas as pd 
import re
from bs4 import BeautifulSoup
from keras.preprocessing.text import Tokenizer 
from keras.preprocessing.sequence import pad_sequences
from nltk.corpus import stopwords
from tensorflow.keras.layers import Input, LSTM, Embedding, Dense, Concatenate, TimeDistributed
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import EarlyStopping
from keras.initializers import Constant
import warnings
pd.set_option("display.max_colwidth", 200)
warnings.filterwarnings("ignore")
num_wordsX = len(word_to_index)
num_wordsy = len(yword_to_index)

print("Size of vocabulary from the w2v model = {}".format(num_wordsX))
MAX_LEN=100
K.clear_session()

latent_dim = 300
embedding_dim=100

# Encoder
encoder_inputs = Input(shape=(MAX_LEN, ))

#embedding layer
enc_emb =  Embedding(input_dim=num_wordsX, 
                     output_dim=embedding_dim,
                     embeddings_initializer=Constant(embedding_matrixX),
                     trainable=True,
                     input_length = MAX_LEN
                     
                     )(encoder_inputs)

#encoder lstm 1
encoder_lstm1 = LSTM(latent_dim,return_sequences=True,return_state=True,dropout=0.4,recurrent_dropout=0.4)
encoder_output1, state_h1, state_c1 = encoder_lstm1(enc_emb)

#encoder lstm 2
encoder_lstm2 = LSTM(latent_dim,return_sequences=True,return_state=True,dropout=0.4,recurrent_dropout=0.4)
encoder_output2, state_h2, state_c2 = encoder_lstm2(encoder_output1)

#encoder lstm 3
encoder_lstm3=LSTM(latent_dim, return_state=True, return_sequences=True,dropout=0.4,recurrent_dropout=0.4)
encoder_outputs, state_h, state_c= encoder_lstm3(encoder_output2)

# Set up the decoder, using `encoder_states` as initial state.
decoder_inputs = Input(shape=(None,))

#embedding layer
dec_emb_layer = Embedding(input_dim=num_wordsy,
                          output_dim=embedding_dim,
                          trainable=True,
                          embeddings_initializer=Constant(embedding_matrixy),
                          input_length=MAX_LEN)
dec_emb = dec_emb_layer(decoder_inputs)

decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True,dropout=0.4,recurrent_dropout=0.2)
decoder_outputs,decoder_fwd_state, decoder_back_state = decoder_lstm(dec_emb,initial_state=[state_h, state_c])

#Attention layer
attn_layer = AttentionLayer(name='attention_layer')
attn_out, attn_states = attn_layer([encoder_outputs, decoder_outputs])

#Concating Attention input and Decoder LSTM output
decoder_concat_input = Concatenate(axis=-1, name='concat_layer')([decoder_outputs, attn_out])

#dense layer
decoder_dense =  TimeDistributed(Dense(num_wordsy, activation='softmax'))
decoder_outputs = decoder_dense(decoder_concat_input)

# Define the model 
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

model.summary()


Size of vocabulary from the w2v model = 70132
Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 100)]        0           []                               
                                                                                                  
 embedding (Embedding)          (None, 100, 100)     7013200     ['input_1[0][0]']                
                                                                                                  
 lstm (LSTM)                    [(None, 100, 300),   481200      ['embedding[0][0]']              
                                 (None, 300),                                                     
                                 (None, 300)]                                                     
                                                

In [ ]:
embedding_matrixX.shape

(70132, 100)

In [ ]:
X_train_encoded.shape

(50000, 100)

In [ ]:
model.compile(optimizer='rmsprop', loss='sparse_categorical_crossentropy')


In [ ]:
es = EarlyStopping(monitor='val_loss', mode='min', verbose=1,patience=2)


## visualize model

In [ ]:
#Visualize the Model
from tensorflow.keras.utils import plot_model

plot_model(model, to_file='model_plot.png', show_shapes=True, show_layer_names=True)

## Train model

In [ ]:
# history=model.fit(x_tr, y_tr,epochs=50,callbacks=[es],batch_size=128, validation_data=(x_val,y_val))

# history=model.fit([X_train_encoded,y_train_encoded[:,:]], y_train_encoded.reshape(y_train_encoded.shape[0],y_train_encoded.shape[1], 1)[:,1:] ,
#                   epochs=50,callbacks=[es],
#                   batch_size=256, 
#                   validation_data=([X_dev_encoded,y_dev_encoded[:,:]], y_dev_encoded.reshape(y_dev_encoded.shape[0],y_dev_encoded.shape[1], 1)[:,1:]))

history=model.fit([X_train_encoded,y_train_encoded[:,:-1]], y_train_encoded.reshape(y_train_encoded.shape[0],y_train_encoded.shape[1], 1)[:,1:] ,
                  epochs=50,callbacks=[es],
                  batch_size=256, 
                  validation_data=([X_dev_encoded,y_dev_encoded[:,:-1]], y_dev_encoded.reshape(y_dev_encoded.shape[0],y_dev_encoded.shape[1], 1)[:,1:]))



Epoch 1/50
196/196 [==============================] - 408s 2s/step - loss: 2.5233 - val_loss: 2.1571
Epoch 2/50
196/196 [==============================] - 398s 2s/step - loss: 2.0693 - val_loss: 1.9132
Epoch 3/50
196/196 [==============================] - 398s 2s/step - loss: 1.9053 - val_loss: 1.8038
Epoch 4/50
196/196 [==============================] - 398s 2s/step - loss: 1.8102 - val_loss: 1.7312
Epoch 5/50
196/196 [==============================] - 398s 2s/step - loss: 1.7464 - val_loss: 1.6869
Epoch 6/50
196/196 [==============================] - 398s 2s/step - loss: 1.6996 - val_loss: 1.6558
Epoch 7/50
196/196 [==============================] - 398s 2s/step - loss: 1.6636 - val_loss: 1.6265
Epoch 8/50
196/196 [==============================] - 398s 2s/step - loss: 1.6327 - val_loss: 1.6077
Epoch 9/50
196/196 [==============================] - 398s 2s/step - loss: 1.6091 - val_loss: 1.5922
Epoch 10/50
196/196 [==============================] - 398s 2s/step - loss: 1.5888 - val_lo

##decode

In [ ]:
reverse_target_word_index=yword_tokenizer.index_word
reverse_source_word_index=word_tokenizer.index_word
target_word_index=yword_tokenizer.word_index

In [ ]:
# Encode the input sequence to get the feature vector
encoder_model = Model(inputs=encoder_inputs,outputs=[encoder_outputs, state_h, state_c])

# Decoder setup
# Below tensors will hold the states of the previous time step
decoder_state_input_h = Input(shape=(latent_dim,))
decoder_state_input_c = Input(shape=(latent_dim,))
decoder_hidden_state_input = Input(shape=(MAX_LEN,latent_dim))

# Get the embeddings of the decoder sequence
dec_emb2= dec_emb_layer(decoder_inputs) 
# To predict the next word in the sequence, set the initial states to the states from the previous time step
decoder_outputs2, state_h2, state_c2 = decoder_lstm(dec_emb2, initial_state=[decoder_state_input_h, decoder_state_input_c])


#Attention inference
attn_out_inf, attn_states_inf = attn_layer([decoder_hidden_state_input, decoder_outputs2])
decoder_inf_concat = Concatenate(axis=-1, name='concat')([decoder_outputs2, attn_out_inf])

#Adding Dense softmax layer to generate proability distribution over the target vocabulary
decoder_outputs2 = decoder_dense(decoder_inf_concat) 

# Final decoder model
decoder_model = Model(
    [decoder_inputs] + [decoder_hidden_state_input,decoder_state_input_h, decoder_state_input_c],
    [decoder_outputs2] + [state_h2, state_c2])

In [ ]:
with open('/content/drive/MyDrive/Năm 3/NLP/Do_An/encoder_50_model.json', 'w', encoding='utf8') as f:
    f.write(encoder_model.to_json())
encoder_model.save_weights('/content/drive/MyDrive/Năm 3/NLP/Do_An/encoder_model_50_seq2seq_vitokenizer.h5')

with open('/content/drive/MyDrive/Năm 3/NLP/Do_An/decoder_50_model.json', 'w', encoding='utf8') as f:
    f.write(decoder_model.to_json())
decoder_model.save_weights('/content/drive/MyDrive/Năm 3/NLP/Do_An/decoder_model_50_seq2seq_vitokenizer.h5')
#/content/drive/MyDrive/Năm 3/NLP/Do_An/dataset/dataset/data_train.csv

**We are defining a function below which is the implementation of the inference process**

In [ ]:
def decode_sequence(input_seq):
    # Encode the input as state vectors.
    e_out, e_h, e_c = encoder_model.predict(input_seq)
    
    # Generate empty target sequence of length 1.
    target_seq = np.zeros((1,1))
    
    # Populate the first word of target sequence with the start word.
    target_seq[0, 0] = target_word_index['sostok']

    stop_condition = False
    decoded_sentence = ''
    while not stop_condition:
      
        output_tokens, h, c = decoder_model.predict([target_seq] + [e_out, e_h, e_c])

        # Sample a token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_token = reverse_target_word_index[sampled_token_index]
        
        if(sampled_token!='eostok'):
            decoded_sentence += ' '+sampled_token

        # Exit condition: either hit max length or find stop word.
        if (sampled_token == 'eostok'  or len(decoded_sentence.split()) >= (100-1)):
            stop_condition = True

        # Update the target sequence (of length 1).
        target_seq = np.zeros((1,1))
        target_seq[0, 0] = sampled_token_index

        # Update internal states
        e_h, e_c = h, c

    return decoded_sentence

**Let us define the functions to convert an integer sequence to a word sequence for summary as well as the reviews:**


In [ ]:
def seq2summary(input_seq):
    newString=''
    for i in input_seq:
        if((i!=0 and i!=target_word_index['sostok']) and i!=target_word_index['eostok']):
            newString=newString+str(reverse_target_word_index[i])+' '
    return newString

def seq2text(input_seq):
    newString=''
    for i in input_seq:
        if(i!=0):
            newString=newString+str(reverse_source_word_index[i])+' '
    return newString

**Run the model over the data to see the results**

### predict

In [ ]:
!pip install rouge-score -qq

In [ ]:
from rouge_score import rouge_scorer
scorer = rouge_scorer.RougeScorer(['rouge1','rouge2', 'rougeL'], use_stemmer=True)
for i in range(0,10):
    print("Review:",seq2text(X_test_encoded[i]))
    print("Original summary:",seq2summary(y_test_encoded[i]))
    print("Predicted summary:",decode_sequence(X_test_encoded[i].reshape(1,100)))
    scores = scorer.score(seq2summary(y_test_encoded[i]), decode_sequence(X_test_encoded[i].reshape(1,100)))
    print(scores)
    print("\n")


NameError: ignored

In [ ]:
set_test.summary[0:10]

0                                                                                Bộ_trưởng Quốc_phòng Anh vạch ra chiến_lược hậu Brexit , đề_cao phô_trương sức_mạnh quân_sự để bảo_vệ lợi_ích của London . 
1    Nhận kỷ_luật sau khi đi lễ trong giờ_hành_chính , Giám_đốc trung_tâm hỗ_trợ xuất_khẩu ( Cục Xúc_tiến thương_mại ) bị hạ bậc lương . Hội_đồng_kỷ_luật cũng cảnh_cáo 7 cán_bộ cấp phòng , khiển_trách ...
2           Theo quy_định của luật Khiếu_nại , các cơ_quan Nhà_nước có thẩm_quyền không được thụ_lý giải_quyết đối_với khiếu_nại đã có quyết_định giải_quyết lần 2 . Việc này thuộc thẩm_quyền của toà_án . 
3                                                                  Ngày mùng 3 Tết , phố Sách Hà_Nội mở_cửa và sẽ kéo_dài tới ngày mùng 10 tế và đây trở_thành một điểm đến của nhiều gia_đình đi du_xuân . 
4                                                                            Một cặp vợ_chồng tại thành_phố Hamburg , Đức đã lơ_đễnh quên mất đứa con vừa chào_đời của họ khi đi tax

In [ ]:
len(set_test.summary)

22644

In [ ]:
from rouge_score import rouge_scorer
#scorer = rouge_scorer.RougeScorer(['rouge1','rouge2', 'rougeL'], use_stemmer=True)
df_predict_test = []
for i in range(len(set_test.summary)):
    # print("Review:",seq2text(X_test_encoded[i]))
    # print("Original summary:",seq2summary(y_test_encoded[i]))
    # print("Predicted summary:",decode_sequence(X_test_encoded[i].reshape(1,100)))
    df_predict_test.append(decode_sequence(X_test_encoded[i].reshape(1,100)))
    #scores = scorer.score(set_test.summary[i], decode_sequence(X_test_encoded[i].reshape(1,100)))
    print(i)

Streaming output truncated to the last 5000 lines.
12794
12795
12796
12797
12798
12799
12800
12801
12802
12803
12804
12805
12806
12807
12808
12809
12810
12811
12812
12813
12814
12815
12816
12817
12818
12819
12820
12821
12822
12823
12824
12825
12826
12827
12828
12829
12830
12831
12832
12833
12834
12835
12836
12837
12838
12839
12840
12841
12842
12843
12844
12845
12846
12847
12848
12849
12850
12851
12852
12853
12854
12855
12856
12857
12858
12859
12860
12861
12862
12863
12864
12865
12866
12867
12868
12869
12870
12871
12872
12873
12874
12875
12876
12877
12878
12879
12880
12881
12882
12883
12884
12885
12886
12887
12888
12889
12890
12891
12892
12893
12894
12895
12896
12897
12898
12899
12900
12901
12902
12903
12904
12905
12906
12907
12908
12909
12910
12911
12912
12913
12914
12915
12916
12917
12918
12919
12920
12921
12922
12923
12924
12925
12926
12927
12928
12929
12930
12931
12932
12933
12934
12935
12936
12937
12938
12939
12940
12941
12942
12943
12944
12945
12946
12947
12948
12949
12950
12951
1

In [ ]:
import pandas as pd

df_predict_test = pd.DataFrame(df_predict_test, columns = ['predict'])

NameError: ignored

In [ ]:
df_predict_test.to_csv('/content/drive/MyDrive/Năm 3/NLP/Do_An/predict_full.csv', encoding='utf-8')

NameError: ignored

In [ ]:
from keras.models import model_from_json

# Load model and Predict

In [ ]:
from keras.models import model_from_json

In [ ]:
def load_model(model_filename, model_weights_filename):
    with open(model_filename, 'r', encoding='utf8') as f:
        model = model_from_json(f.read(), custom_objects={'AttentionLayer': AttentionLayer})
    model.load_weights(model_weights_filename)
    return model

encoder = load_model('/content/drive/MyDrive/Năm 3/NLP/Do_An/encoder_50_model.json', '/content/drive/MyDrive/Năm 3/NLP/Do_An/encoder_model_50_seq2seq_vitokenizer.h5')
decoder = load_model('/content/drive/MyDrive/Năm 3/NLP/Do_An/decoder_50_model.json', '/content/drive/MyDrive/Năm 3/NLP/Do_An/decoder_model_50_seq2seq_vitokenizer.h5')

In [ ]:
reverse_target_word_index=yword_tokenizer.index_word
reverse_source_word_index=word_tokenizer.index_word
target_word_index=yword_tokenizer.word_index

In [ ]:
def decode_sequence(input_seq):
    # Encode the input as state vectors.
    e_out, e_h, e_c = encoder.predict(input_seq)
    
    # Generate empty target sequence of length 1.
    target_seq = np.zeros((1,1))
    
    # Populate the first word of target sequence with the start word.
    target_seq[0, 0] = target_word_index['sostok']

    stop_condition = False
    decoded_sentence = ''
    while not stop_condition:
      
        output_tokens, h, c = decoder.predict([target_seq] + [e_out, e_h, e_c])

        # Sample a token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_token = reverse_target_word_index[sampled_token_index]
        
        if(sampled_token!='eostok'):
            decoded_sentence += ' '+str(sampled_token)

        # Exit condition: either hit max length or find stop word.
        if (sampled_token == 'eostok'  or len(decoded_sentence.split()) >= (100-1)):
            stop_condition = True

        # Update the target sequence (of length 1).
        target_seq = np.zeros((1,1))
        target_seq[0, 0] = sampled_token_index

        # Update internal states
        e_h, e_c = h, c

    return decoded_sentence

In [ ]:
def seq2summary(input_seq):
    newString=''
    for i in input_seq:
        if((i!=0 and i!=target_word_index['sostok']) and i!=target_word_index['eostok']):
            newString=newString+str(reverse_target_word_index[i])+' '
    return newString

def seq2text(input_seq):
    newString=''
    for i in input_seq:
        if(i!=0):
            newString=newString+str(reverse_source_word_index[i])+' '
    return newString

In [ ]:
!pip install -q rouge_score

In [ ]:
from rouge_score import rouge_scorer
scorer = rouge_scorer.RougeScorer(['rouge1','rouge2', 'rougeL'], use_stemmer=True)
for i in range(0,10):
    print("Review:",seq2text(X_test_encoded[i]))
    print("Original summary:",seq2summary(y_test_encoded[i]))
    print("Predicted summary:",decode_sequence(X_test_encoded[i].reshape(1,100)))
    scores = scorer.score(seq2summary(y_test_encoded[i]), decode_sequence(X_test_encoded[i].reshape(1,100)))
    print(scores)
    print("\n")

Review: kiến được trang bị hệ thống phóng và cáp hãm đà cho máy bay như lớp nimitz và gerald r ford của mỹ nhưng chi phí quá cao buộc hải quân anh lựa chọn giải pháp stovl và sử dụng tiêm kích f 35 b hms queen elizabeth có khả năng chở tối đa 36 máy bay f 35 b và 4 trực thăng cảnh báo sớm crowsnest nó có thể được bổ sung tới 12 trực thăng chinook hoặc merlin và 8 trực thăng tấn công ah 64 apache hms queen elizabeth thử nghiệm trên biển hồi năm 2018 
Original summary: bộ trưởng quốc phòng anh vạch ra chiến lược hậu brexit đề cao phô trương sức mạnh quân sự để bảo vệ lợi ích của london 
Predicted summary:  cho giá sát thiệt nguyễn ông quốc được một là đã có phát làm này với các nhà giá sát không người ma xe giá sát vào ôtô can quân sự ở lệnh nam nước việc
{'rouge1': Score(precision=0.41509433962264153, recall=0.5789473684210527, fmeasure=0.4835164835164836), 'rouge2': Score(precision=0.1346153846153846, recall=0.1891891891891892, fmeasure=0.15730337078651685), 'rougeL': Score(precision=0

InvalidArgumentError: ignored

In [ ]:
list_pred = []
for i in range(len(X_test_encoded)): 
    print("Predicted summary:", i)
    list_pred.append(decode_sequence(X_test_encoded[i].reshape(1,100)))


Streaming output truncated to the last 5000 lines.
Predicted summary: 15000
Predicted summary: 15001
Predicted summary: 15002
Predicted summary: 15003
Predicted summary: 15004
Predicted summary: 15005
Predicted summary: 15006
Predicted summary: 15007
Predicted summary: 15008
Predicted summary: 15009
Predicted summary: 15010
Predicted summary: 15011
Predicted summary: 15012
Predicted summary: 15013
Predicted summary: 15014
Predicted summary: 15015
Predicted summary: 15016
Predicted summary: 15017
Predicted summary: 15018
Predicted summary: 15019
Predicted summary: 15020
Predicted summary: 15021
Predicted summary: 15022
Predicted summary: 15023
Predicted summary: 15024
Predicted summary: 15025
Predicted summary: 15026
Predicted summary: 15027
Predicted summary: 15028
Predicted summary: 15029
Predicted summary: 15030
Predicted summary: 15031
Predicted summary: 15032
Predicted summary: 15033
Predicted summary: 15034
Predicted summary: 15035
Predicted summary: 15036
Predicted summary: 15037

In [ ]:
data_pred = pd.DataFrame(list_pred, columns = ['Predict'])

In [ ]:
data_pred.shape

(22643, 1)

In [ ]:
len(df_test)

20000

In [ ]:
bigdata = pd.concat([data_pred, df_test.summary], ignore_index=True, sort=False)

In [ ]:
bigdata

,Predict,0
0,phi đội mỹ đang điều tra bổ sung các phi công tiêm kích f 35 a poseidon của mỹ để tìm kiếm chiếc máy bay b 52 của mỹ,NaN
1,lãnh đạo bộ vh tt dl vừa có văn bản gửi bộ vh tt dl cho biết đơn vị này sẽ có văn bản gửi bộ vh tt dl cho biết sẽ có văn bản gửi bộ vh tt dl cho biết sẽ có văn bản gửi bộ vh tt dl cho biết sẽ có ...,NaN
2,sau khi bị cáo buộc phải bồi thường cho vinasun ông lê thị phương thảo nguyên giám đốc sở hữu nghị tp hcm nguyễn thiện nhân cho biết đã có đơn kháng cáo buộc phải bồi thường cho vinasun án sơ thẩm,NaN
3,chiều 19 5 tại tp hcm đã tổ chức lễ hội mùa hè năm 2019 tại tp hcm,NaN
4,sau khi va chạm với xe khách giường nằm trong khuôn viên taxi ở hà tĩnh đã bị xe khách giường nằm trong cabin,NaN
...,...,...
42638,NaN,"sostok Nổi_tiếng sau “ Thiên_long bát bộ ” , Hồ_Quân , Cao_Hổ , Tu_Khánh , Lâm_Chí_Dĩnh , Lưu_Diệc_Phi tuy cùng xuất_phát_điểm nhưng nhiều người trong số họ có những ngã rẽ khác_biệt . eostok"
42639,NaN,"sostok Bộ Tài_chính đang lấy ý_kiến về Dự_thảo Luật Quản_lý Thuế ( sửa_đổi ) , trong đó có nội_dung cơ_quan thuế sẽ thực_hiện thu thuế đồng_thời thu cả tiền đóng BHXH bắt_buộc của chủ sử_dụng lao_..."
42640,NaN,"sostok Trung_Quốc và Triều_Tiên được cho là đã thảo_luận về những điều_khoản chi_tiết nhằm chấm_dứt vĩnh_viễn chương_trình hạt_nhân và tên_lửa của Bình_Nhưỡng , trong đó có việc viện_trợ tài_chính..."
42641,NaN,sostok Doanh_nghiệp dự_án BOT Thái_Nguyên - Chợ Mới có khả_năng lâm vào tình_trạng vỡ_nợ do chưa được thu phí dự_án sau nửa năm thông xe . eostok


In [ ]:
data_pred.to_csv('/content/drive/MyDrive/Năm 3/NLP/Do_An/predict_full.csv', encoding='utf-8')

In [ ]:
!pip install rouge_score

In [ ]:
from rouge_score import rouge_scorer
scorer = rouge_scorer.RougeScorer(['rouge1','rouge2', 'rougeL'], use_stemmer=True)
list_score = []
for i in range(0,len(set_test)):
    scores = scorer.score(bigdata.iloc[:,0][i],bigdata.iloc[:,1][i] )
    list_score.append(scores)
    print(scores)

AttributeError: ignored

In [ ]:
list_score_r1 = []
list_score_r2 = []
list_score_rl = []
for i in range(10000):
  list_score_r1.append(list_score[i]['rouge1'].fmeasure)
  list_score_r2.append(list_score[i]['rouge2'].fmeasure)
  list_score_rl.append(list_score[i]['rougeL'].fmeasure)

IndexError: ignored

In [ ]:
np.mean(list_score_r1)

In [ ]:
np.mean(list_score_r2)

In [ ]:
np.mean(list_score_rl)

In [ ]:
import pandas as pd
data_pred = pd.read_csv('/content/drive/MyDrive/Năm 3/NLP/Do_An/predict_full.csv')
data_pred.head(5)

,Unnamed: 0,Predict
0,0,phi đội mỹ đang điều tra bổ sung các phi công tiêm kích f 35 a poseidon của mỹ để tìm kiếm chiếc máy bay b 52 của mỹ
1,1,lãnh đạo bộ vh tt dl vừa có văn bản gửi bộ vh tt dl cho biết đơn vị này sẽ có văn bản gửi bộ vh tt dl cho biết sẽ có văn bản gửi bộ vh tt dl cho biết sẽ có văn bản gửi bộ vh tt dl cho biết sẽ có ...
2,2,sau khi bị cáo buộc phải bồi thường cho vinasun ông lê thị phương thảo nguyên giám đốc sở hữu nghị tp hcm nguyễn thiện nhân cho biết đã có đơn kháng cáo buộc phải bồi thường cho vinasun án sơ thẩm
3,3,chiều 19 5 tại tp hcm đã tổ chức lễ hội mùa hè năm 2019 tại tp hcm
4,4,sau khi va chạm với xe khách giường nằm trong khuôn viên taxi ở hà tĩnh đã bị xe khách giường nằm trong cabin


In [ ]:
data_pred.shape

(22643, 2)

In [ ]:
df_test.iloc[:,1][1]

'sostok Nhận kỷ_luật sau khi đi lễ trong giờ_hành_chính , Giám_đốc trung_tâm hỗ_trợ xuất_khẩu ( Cục Xúc_tiến thương_mại ) bị hạ bậc lương . Hội_đồng_kỷ_luật cũng cảnh_cáo 7 cán_bộ cấp phòng , khiển_trách 2 viên_chức .  eostok'

In [ ]:
data_pred.iloc[:1]

,Unnamed: 0,Predict
0,0,phi đội mỹ đang điều tra bổ sung các phi công tiêm kích f 35 a poseidon của mỹ để tìm kiếm chiếc máy bay b 52 của mỹ


In [ ]:
from rouge_score import rouge_scorer
scorer = rouge_scorer.RougeScorer(['rouge1','rouge2', 'rougeL'], use_stemmer=True)
list_score = []
for i in range(0,len(df_test)):
    scores = scorer.score(data_pred.iloc[:,1][i],df_test.iloc[:,1][i] )
    list_score.append(scores)
    print(scores)

Streaming output truncated to the last 5000 lines.
{'rouge1': Score(precision=0.36065573770491804, recall=0.55, fmeasure=0.4356435643564357), 'rouge2': Score(precision=0.11666666666666667, recall=0.1794871794871795, fmeasure=0.14141414141414144), 'rougeL': Score(precision=0.22950819672131148, recall=0.35, fmeasure=0.2772277227722772)}
{'rouge1': Score(precision=0.2916666666666667, recall=0.7, fmeasure=0.4117647058823529), 'rouge2': Score(precision=0.056338028169014086, recall=0.13793103448275862, fmeasure=0.07999999999999999), 'rougeL': Score(precision=0.16666666666666666, recall=0.4, fmeasure=0.23529411764705882)}
{'rouge1': Score(precision=0.28, recall=0.42424242424242425, fmeasure=0.3373493975903614), 'rouge2': Score(precision=0.04081632653061224, recall=0.0625, fmeasure=0.04938271604938272), 'rougeL': Score(precision=0.16, recall=0.24242424242424243, fmeasure=0.19277108433734938)}
{'rouge1': Score(precision=0.5396825396825397, recall=0.6938775510204082, fmeasure=0.6071428571428572)

In [ ]:
list_score_r1 = []
list_score_r2 = []
list_score_rl = []
for i in range(len(df_test)):
  list_score_r1.append(list_score[i]['rouge1'].fmeasure)
  list_score_r2.append(list_score[i]['rouge2'].fmeasure)
  list_score_rl.append(list_score[i]['rougeL'].fmeasure)

In [ ]:
np.mean(list_score_r1)

0.45711662063501446

In [ ]:
np.mean(list_score_r2)

0.11010588993255684

In [ ]:
np.mean(list_score_rl)

0.2721475812184692

In [ ]:
list_score_r1_pre = []
list_score_r2_pre = []
list_score_rl_pre = []
for i in range(len(df_test)):
  list_score_r1_pre.append(list_score[i]['rouge1'].precision)
  list_score_r2_pre.append(list_score[i]['rouge2'].precision)
  list_score_rl_pre.append(list_score[i]['rougeL'].precision)

In [ ]:
np.mean(list_score_r1_pre)

0.4268581959388167

In [ ]:
np.mean(list_score_r2_pre)

0.10221234838706303

In [ ]:
np.mean(list_score_rl_pre)

0.2543479094592381

In [ ]:
list_score_r1_recall = []
list_score_r2_recall = []
list_score_rl_recall = []
for i in range(len(df_test)):
  list_score_r1_recall.append(list_score[i]['rouge1'].recall)
  list_score_r2_recall.append(list_score[i]['rouge2'].recall)
  list_score_rl_recall.append(list_score[i]['rougeL'].recall)

In [ ]:
print(np.mean(list_score_r1_recall))
print(np.mean(list_score_r2_recall))
print(np.mean(list_score_rl_recall))

0.5217922736914066
0.12653359170524897
0.3112957617456655
